## Cluster-Mass based Permutation testing in Julia

- Step 1: load and check adjacency matrix
- Step 2: generate simuated data
- Step 3: run cluster-mass permutation test

### Load Libraries

In [1]:
import Pkg
Pkg.activate("..")
#Pkg.add("CairoMakie")

using UnfoldSim
using MAT
using CairoMakie
using LinearAlgebra

include("../read_adjacency.jl")


  Activating project at `/data/tu_grossmannr/PhDProject_Experimental/ClusterComputation`


read_fieldtrip_neighbors

In [ ]:
# load Hartmut head model
# needed for visualization of adjacency and simulation of effects
hart = Hartmut()

# define custom channels
custom = ["Fp1", "Fp2", "F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2", "F7", "F8", "T7", "T8", "P7", "P8", "AFz", "Fz", "Cz", 
            "Pz", "FC1", "FC2", "CP1", "CP2", "FC5", "FC6", "CP5", "CP6", "FT9", "FT10", "F1", "F2", "C1", "C2", "P1", "P2", 
            "AF3", "AF4", "FC3", "FC4", "CP3", "CP4", "PO3", "PO4", "F5", "F6", "C5", "C6", "P5", "P6", "AF7", "AF8", "FT7", 
            "FT8", "TP7", "TP8", "PO7", "PO8", "Fpz", "CPz", "F9", "F10"]

Please cite: HArtMuT: Harmening Nils, Klug Marius, Gramann Klaus and Miklody Daniel - 10.1088/1741-2552/aca8ce


62-element Vector{String}:
 "Fp1"
 "Fp2"
 "F3"
 "F4"
 "C3"
 "C4"
 "P3"
 "P4"
 "O1"
 "O2"
 ⋮
 "FT8"
 "TP7"
 "TP8"
 "PO7"
 "PO8"
 "Fpz"
 "CPz"
 "F9"
 "F10"

In [3]:
# load custom channel topo
subject_id = "013"
custom_file = "../data/sub-" * subject_id * "_ses-mecha_task-NT_channel_adjacency.mat"

adjacency, matched_labels = read_fieldtrip_neighbors(
    custom_file;
    channel_order = custom  # Reorder to match layout (should not matter in this example, since we use all channels)
)

(Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], ["Fp1", "Fp2", "F3", "F4", "C3", "C4", "P3", "P4", "O1", "O2"  …  "FT7", "FT8", "TP7", "TP8", "PO7", "PO8", "Fpz", "CPz", "F9", "F10"])

In [ ]:
# Visualize the adjacency

# Find indices of channels in HArtMuT
all_labels = hart.electrodes["label"]
channel_idx_raw = [findfirst(==(ch), all_labels) for ch in custom]

# Filter out channels not found in HArtMuT and get their labels
found_mask = .!isnothing.(channel_idx_raw)
channel_idx = Int64[idx for idx in channel_idx_raw if !isnothing(idx)]
electrode_labels = custom[found_mask]


println("Found $(length(electrode_labels)) electrodes out of $(length(custom)) requested")

# Extract subset of electrode positions
electrode_positions = hart.electrodes["pos"][channel_idx, :]
found_indices_in_custom = findall(found_mask)

# Extract the submatrix for only the found channels
adjacency_subset = adjacency[found_indices_in_custom, found_indices_in_custom]

fig = Figure(size = (900, 700))
ax = Axis3(fig[1, 1],
    title = "Electrode Adjacency",
    xlabel = "X", ylabel = "Y", zlabel = "Z",
    aspect = :data)

pos3d = electrode_positions

# Draw connections between adjacent electrodes
for i in 1:size(adjacency_subset, 1)
    for j in (i+1):size(adjacency_subset, 2)
        if adjacency_subset[i, j]
            lines!(ax,
                [pos3d[i, 1], pos3d[j, 1]],
                [pos3d[i, 2], pos3d[j, 2]],
                [pos3d[i, 3], pos3d[j, 3]],
                color = (:gray, 0.5), linewidth = 1.0)
        end
    end
end

# Draw electrode points with labels
scatter!(ax, pos3d[:, 1], pos3d[:, 2], pos3d[:, 3],
    markersize = 12, color = :steelblue)

# Add electrode labels
for (i, label) in enumerate(electrode_labels)
    text!(ax, pos3d[i, 1], pos3d[i, 2], pos3d[i, 3] + 0.01,
        text = label, fontsize = 8, align = (:center, :bottom))
end

fig

Found 60 electrodes out of 62 requested


60×3 Matrix{Float64}:
 -0.0921053   0.235549   -0.00625
  0.0921053   0.235549   -0.00625
 -0.150585    0.145954    0.1125
  0.150585    0.145954    0.1125
 -0.197368   -0.0346821   0.16375
  0.197368   -0.0346821   0.16375
 -0.163743   -0.232659    0.14375
  0.163743   -0.232659    0.14375
 -0.0906433  -0.330925    0.01875
  0.0906433  -0.330925    0.01875
  ⋮                      
  0.162281    0.184971   -0.0075
 -0.232456    0.0346821  -0.005
  0.232456    0.0346821  -0.005
 -0.247076   -0.124277    0.0
  0.247076   -0.124277    0.0
 -0.168129   -0.278902    0.0125
  0.168129   -0.278902    0.0125
  0.0         0.25       -0.00125
  0.0        -0.143064    0.25